In [1]:
import itertools
from pprint import pprint
from qiskit import QuantumCircuit, Aer, execute
backend = Aer.get_backend('statevector_simulator')  
import numpy as np

maxgc = 4
gs = ['h','ccx']
qbnos = 4
qb = list(range(0,qbnos))

C:\Users\DELL\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\DELL\anaconda3\lib\site-packages\numpy\.libs\libopenblas.jpijnswnnan3ce6lli5fwsphut2vxmth.gfortran-win_amd64.dll
C:\Users\DELL\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
qb

[0, 1, 2, 3]

In [3]:
def toStr(n,base):
    '''
    Convert a decimal number to base-n
    '''
    convertString = "0123456789ABCDEF"
    if n < base:
        return convertString[n]
    else:
        return toStr(n//base,base) + convertString[n%base]

In [4]:
toStr(15,16)

'F'

In [5]:
def H_choice(opcode):
    c = list(itertools.combinations(qb, 1))
    #print(c)
    allc = []
    for i in c:
        allc.append(opcode+str(i[0]))
    return allc

In [6]:
def CCX_choice(opcode):
    c = list(itertools.combinations(qb, 3))
    allc = []
    #print(c)
    for i in c:
        allc.append(opcode+str(i[0])+str(i[1])+str(i[2]))
        allc.append(opcode+str(i[1])+str(i[2])+str(i[0]))
        allc.append(opcode+str(i[2])+str(i[0])+str(i[1]))
    return allc

In [7]:
setH = H_choice('0')
setCCX = CCX_choice('1')

In [8]:
setH

['00', '01', '02', '03']

In [9]:
setCCX

['1012',
 '1120',
 '1201',
 '1013',
 '1130',
 '1301',
 '1023',
 '1230',
 '1302',
 '1123',
 '1231',
 '1312']

In [10]:
def progs(gc):
    p = []
    for i in range(0,len(gs)**gc):
        gseq = toStr(i,len(gs)).zfill(gc)
        #print(gseq)
        cg = ['']          
        for j in range(0, len(gseq)):
            if gseq[j] == '0':
                g = list(itertools.product(cg, setH))
            if gseq[j] == '1':
                g = list(itertools.product(cg, setCCX))
            cg = []
            for k in g:
                cg.append(''.join(map(str, k)))
        #print(gseq,': ',cg)
        for j in cg:
            p.append(j)
    return p

In [11]:
progs(1)

['00',
 '01',
 '02',
 '03',
 '1012',
 '1120',
 '1201',
 '1013',
 '1130',
 '1301',
 '1023',
 '1230',
 '1302',
 '1123',
 '1231',
 '1312']

In [12]:
def state_construction(index,qc):
    for i in range(4):
        if(index[i]) == '1':
            qc.x(i)

In [13]:
possible_states = list(itertools.product('01', repeat=4))
possible_states

[('0', '0', '0', '0'),
 ('0', '0', '0', '1'),
 ('0', '0', '1', '0'),
 ('0', '0', '1', '1'),
 ('0', '1', '0', '0'),
 ('0', '1', '0', '1'),
 ('0', '1', '1', '0'),
 ('0', '1', '1', '1'),
 ('1', '0', '0', '0'),
 ('1', '0', '0', '1'),
 ('1', '0', '1', '0'),
 ('1', '0', '1', '1'),
 ('1', '1', '0', '0'),
 ('1', '1', '0', '1'),
 ('1', '1', '1', '0'),
 ('1', '1', '1', '1')]

In [14]:
def conv2QP(desc):
    qc_list = []
    for j in possible_states:
        qcirc = QuantumCircuit(qbnos,qbnos)
        state_construction(j,qcirc)
        i = 0
        while (i < len(desc)):
            if desc[i]=='0':
                # print('H',desc[i+1])
                qcirc.h(int(desc[i+1]))
                i+= 2
            elif desc[i]=='1':
                # print('CCX',desc[i+1],desc[i+2],desc[i+3])
                qcirc.ccx(int(desc[i+1]),int(desc[i+2]),int(desc[i+3]))
                i+= 4
        qcirc.barrier()
        qc_list.append(qcirc)
    return qc_list

In [15]:
def runQprog(desc):
    mem_result= []
    qcirc = conv2QP(desc)
    # print(desc)
    # print(qcirc.draw())
    for i in qcirc:
        job = execute(i, backend, shots=1, memory=True)
        result = job.result()
        memory = result.get_statevector(i).probabilities()
        mem_result.append(memory)
    return mem_result

## Test for one initial case

In [16]:
def conv2QP_one_intial(desc, state):
    qcirc = QuantumCircuit(qbnos,qbnos)
    state_construction(state,qcirc)
    i = 0
    while (i < len(desc)):
        if desc[i]=='0':
            # print('X',desc[i+1])
            qcirc.h(int(desc[i+1]))
            i+= 2
        elif desc[i]=='1':
            # print('CCX',desc[i+1],desc[i+2],desc[i+3])
            qcirc.ccx(int(desc[i+1]),int(desc[i+2]),int(desc[i+3]))
            i+= 4
    qcirc.barrier()
#         for i in range(0,qbnos):
#             qcirc.measure(i,i)
    return qcirc

In [17]:
def runQprog_one_intial(desc,state):
    qcirc = conv2QP_one_intial(desc,state)
    # print(desc)
    # print(qcirc.draw())
    job = execute(qcirc, backend, shots=1, memory=True)
    result = job.result()
    memory = result.get_statevector()
    return memory

In [18]:
runQprog_one_intial('03','0011').probabilities()

array([0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5,
       0. , 0. , 0. ])

In [19]:
gc = 1
ng = {}
p = progs(gc)
for i in p:
    op = runQprog_one_intial(i,'0000')
    ng[i] = op
print("nGC Level",gc)
# for key, value in sorted(ng.items()):
#     value.sort(key=len)
#     print(key, len(value),value[0],len(value[0]))

nGC Level 1


In [32]:
two_gate = progs(1)

A -> B (layer 1)
A -> -> B (layer 2)

In [36]:
l = []
for i in (two_gate):
    l.append(ng[i].probabilities())

In [37]:
l

[array([0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ]),
 array([0.5, 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ]),
 array([0.5, 0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ]),
 array([0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ]),
 array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 

## For all cases

In [60]:
gc = 1
ng_all = {}
p = progs(gc)
for i in p:
    op = runQprog(i)
    ng_all[i] = op
print("nGC Level",gc)
# for key, value in sorted(ng.items()):
#     value.sort(key=len)
#     print(key, len(value),value[0],len(value[0]))

nGC Level 1


In [61]:
one_gate = progs(1)

In [62]:
l_all = []
for i in (one_gate):
    l_all.append(ng_all[i])

In [63]:
l_all

[[array([0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
         0. , 0. , 0. ]),
  array([0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. ,
         0. , 0. , 0. ]),
  array([0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
         0. , 0. , 0. ]),
  array([0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5,
         0.5, 0. , 0. ]),
  array([0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
         0. , 0. , 0. ]),
  array([0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. ,
         0. , 0. , 0. ]),
  array([0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. ,
         0. , 0. , 0. ]),
  array([0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
         0. , 0.5, 0.5]),
  array([0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
         0. , 0. , 0. ]),
  array([0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. ,
         0. , 0. , 0. ]),


In [64]:
possible_states = ['0000',
  '0001',
  '0010',
  '0011',
  '0100',
  '0101',
  '0110',
  '0111',
  '1000',
  '1001',
  '1010',
  '1011',
  '1100',
  '1101',
  '1110',
  '1111']

In [65]:
# short circuits at shortest nested list if table is jagged:
list(map(list, zip(*l_all)))

# discards no data if jagged and fills short nested lists with None
transpose_list = list(map(list, itertools.zip_longest(*l_all, fillvalue=None)))

### List of all first layer 

In [66]:
transpose_list

[[array([0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
         0. , 0. , 0. ]),
  array([0.5, 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
         0. , 0. , 0. ]),
  array([0.5, 0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
         0. , 0. , 0. ]),
  array([0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. ,
         0. , 0. , 0. ]),
  array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),


In [67]:
list_count = np.zeros([16,16])

In [68]:
for i in range (qbnos**2):
    for j in transpose_list[i]:
        list_count[i] += j

In [69]:
list_count

array([[14. ,  0.5,  0.5,  0. ,  0.5,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 14. ,  0.5,  0.5,
         0. ,  0.5,  0. ,  0. ,  0. ],
       [ 0.5,  0. ,  0. ,  0. , 14. ,  0.5,  0.5,  0. ,  0. ,  0. ,  0. ,
         0. ,  0.5,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,
         0. , 12. ,  1.5,  1.5,  0. ],
       [ 0.5,  0. , 14. ,  0.5,  0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0.5,
         0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. , 12. ,
         1.5,  0. ,  0. ,  1.5,  0. ],
       [ 0. ,  0. ,  0.5,  0. ,  0.5,  0. , 12. ,  1.5,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  1.5,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1.5,  0. ,  0. ,  0. ,  1.5,
         0. ,  1.5,  0. ,  8. ,  3.5],
       [ 0.5, 14. ,  0. ,  0.5,  0. ,  0.5,  0. ,  0. ,  0. ,  0.5,  0. ,
         0. ,  0. ,  0

In [70]:
l1 = np.array([[14. ,  0.5,  0.5,  0. ,  0.5,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 14. ,  0.5,  0.5,
         0. ,  0.5,  0. ,  0. ,  0. ],
       [ 0.5,  0. ,  0. ,  0. , 14. ,  0.5,  0.5,  0. ,  0. ,  0. ,  0. ,
         0. ,  0.5,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,
         0. , 12. ,  1.5,  1.5,  0. ],
       [ 0.5,  0. , 14. ,  0.5,  0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0.5,
         0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. , 12. ,
         1.5,  0. ,  0. ,  1.5,  0. ],
       [ 0. ,  0. ,  0.5,  0. ,  0.5,  0. , 12. ,  1.5,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  1.5,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1.5,  0. ,  0. ,  0. ,  1.5,
         0. ,  1.5,  0. ,  8. ,  3.5],
       [ 0.5, 14. ,  0. ,  0.5,  0. ,  0.5,  0. ,  0. ,  0. ,  0.5,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5, 12. ,  0. ,
         1.5,  0. ,  1.5,  0. ,  0. ],
       [ 0. ,  0.5,  0. ,  0. ,  0.5, 12. ,  0. ,  1.5,  0. ,  0. ,  0. ,
         0. ,  0. ,  1.5,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  1.5,  0. ,  0. ,  0. ,  1.5,  0. ,
         0. ,  1.5,  8. ,  0. ,  3.5],
       [ 0. ,  0.5,  0.5, 12. ,  0. ,  0. ,  0. ,  1.5,  0. ,  0. ,  0. ,
         1.5,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  1.5,  0. ,  0. ,  0. ,  0. ,  0. ,  1.5,  1.5,
         8. ,  0. ,  0. ,  0. ,  3.5],
       [ 0. ,  0. ,  0. ,  1.5,  0. ,  1.5,  1.5,  8. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  3.5],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  3.5,  0. ,  0. ,  0. ,
         3.5,  0. ,  3.5,  3.5,  2. ]])

In [71]:
l2 = np.array([[199. ,  13.5,  13.5,   0.5,  13.5,   0.5,   0.5,   0. ,  13.5,
          0.5,   0.5,   0. ,   0.5,   0. ,   0. ,   0. ],
       [ 13.5,   0.5,   0.5,   0. ,   0.5,   0. ,   0. ,   0. , 199. ,
         12.5,  12.5,   1.5,  12.5,   1.5,   1.5,   0. ],
       [ 13.5,   0.5,   0.5,   0. , 199. ,  12.5,  12.5,   1.5,   0.5,
          0. ,   0. ,   0. ,  12.5,   1.5,   1.5,   0. ],
       [  0.5,   0. ,   0. ,   0. ,  12.5,   2.5,   2.5,   0. ,  12.5,
          2.5,   2.5,   0. , 151. ,  29.5,  29.5,  10.5],
       [ 13.5,   0.5, 199. ,  12.5,   0.5,   0. ,  12.5,   1.5,   0.5,
          0. ,  12.5,   1.5,   0. ,   0. ,   1.5,   0. ],
       [  0.5,   0. ,  12.5,   2.5,   0. ,   0. ,   2.5,   0. ,  12.5,
          2.5, 151. ,  29.5,   2.5,   0. ,  29.5,  10.5],
       [  0.5,   0. ,  12.5,   2.5,  12.5,   2.5, 151. ,  29.5,   0. ,
          0. ,   2.5,   0. ,   2.5,   0. ,  29.5,  10.5],
       [  0. ,   0. ,   1.5,   0. ,   1.5,   0. ,  29.5,  14.5,   1.5,
          0. ,  29.5,  14.5,  29.5,  14.5,  85. ,  34.5],
       [ 13.5, 199. ,   0.5,  12.5,   0.5,  12.5,   0. ,   1.5,   0.5,
         12.5,   0. ,   1.5,   0. ,   1.5,   0. ,   0. ],
       [  0.5,  12.5,   0. ,   2.5,   0. ,   2.5,   0. ,   0. ,  12.5,
        151. ,   2.5,  29.5,   2.5,  29.5,   0. ,  10.5],
       [  0.5,  12.5,   0. ,   2.5,  12.5, 151. ,   2.5,  29.5,   0. ,
          2.5,   0. ,   0. ,   2.5,  29.5,   0. ,  10.5],
       [  0. ,   1.5,   0. ,   0. ,   1.5,  29.5,   0. ,  14.5,   1.5,
         29.5,   0. ,  14.5,  29.5,  85. ,  14.5,  34.5],
       [  0.5,  12.5,  12.5, 151. ,   0. ,   2.5,   2.5,  29.5,   0. ,
          2.5,   2.5,  29.5,   0. ,   0. ,   0. ,  10.5],
       [  0. ,   1.5,   1.5,  29.5,   0. ,   0. ,   0. ,  14.5,   1.5,
         29.5,  29.5,  85. ,   0. ,  14.5,  14.5,  34.5],
       [  0. ,   1.5,   1.5,  29.5,   1.5,  29.5,  29.5,  85. ,   0. ,
          0. ,   0. ,  14.5,   0. ,  14.5,  14.5,  34.5],
       [  0. ,   0. ,   0. ,  10.5,   0. ,  10.5,  10.5,  34.5,   0. ,
         10.5,  10.5,  34.5,  10.5,  34.5,  34.5,  55. ]])

In [72]:
l3 = np.array([[2.8670e+03, 2.7725e+02, 2.7725e+02, 1.8500e+01, 2.7725e+02,
        1.8500e+01, 1.8500e+01, 2.2500e+00, 2.7725e+02, 1.8500e+01,
        1.8500e+01, 2.2500e+00, 1.8500e+01, 2.2500e+00, 2.2500e+00,
        0.0000e+00],
       [2.7725e+02, 1.9000e+01, 1.9000e+01, 2.7500e+00, 1.9000e+01,
        2.7500e+00, 2.7500e+00, 0.0000e+00, 2.8625e+03, 2.4325e+02,
        2.4325e+02, 4.8500e+01, 2.4325e+02, 4.8500e+01, 4.8500e+01,
        1.5750e+01],
       [2.7725e+02, 1.9000e+01, 1.9000e+01, 2.7500e+00, 2.8625e+03,
        2.4325e+02, 2.4325e+02, 4.8500e+01, 1.9000e+01, 2.7500e+00,
        2.7500e+00, 0.0000e+00, 2.4325e+02, 4.8500e+01, 4.8500e+01,
        1.5750e+01],
       [1.8500e+01, 2.7500e+00, 2.7500e+00, 0.0000e+00, 2.4325e+02,
        7.9000e+01, 7.9000e+01, 4.4250e+01, 2.4325e+02, 7.9000e+01,
        7.9000e+01, 4.4250e+01, 1.9550e+03, 5.0075e+02, 5.0075e+02,
        2.2450e+02],
       [2.7725e+02, 1.9000e+01, 2.8625e+03, 2.4325e+02, 1.9000e+01,
        2.7500e+00, 2.4325e+02, 4.8500e+01, 1.9000e+01, 2.7500e+00,
        2.4325e+02, 4.8500e+01, 2.7500e+00, 0.0000e+00, 4.8500e+01,
        1.5750e+01],
       [1.8500e+01, 2.7500e+00, 2.4325e+02, 7.9000e+01, 2.7500e+00,
        0.0000e+00, 7.9000e+01, 4.4250e+01, 2.4325e+02, 7.9000e+01,
        1.9550e+03, 5.0075e+02, 7.9000e+01, 4.4250e+01, 5.0075e+02,
        2.2450e+02],
       [1.8500e+01, 2.7500e+00, 2.4325e+02, 7.9000e+01, 2.4325e+02,
        7.9000e+01, 1.9550e+03, 5.0075e+02, 2.7500e+00, 0.0000e+00,
        7.9000e+01, 4.4250e+01, 7.9000e+01, 4.4250e+01, 5.0075e+02,
        2.2450e+02],
       [2.2500e+00, 0.0000e+00, 4.8500e+01, 4.4250e+01, 4.8500e+01,
        4.4250e+01, 5.0075e+02, 2.7700e+02, 4.8500e+01, 4.4250e+01,
        5.0075e+02, 2.7700e+02, 5.0075e+02, 2.7700e+02, 9.6050e+02,
        5.2175e+02],
       [2.7725e+02, 2.8625e+03, 1.9000e+01, 2.4325e+02, 1.9000e+01,
        2.4325e+02, 2.7500e+00, 4.8500e+01, 1.9000e+01, 2.4325e+02,
        2.7500e+00, 4.8500e+01, 2.7500e+00, 4.8500e+01, 0.0000e+00,
        1.5750e+01],
       [1.8500e+01, 2.4325e+02, 2.7500e+00, 7.9000e+01, 2.7500e+00,
        7.9000e+01, 0.0000e+00, 4.4250e+01, 2.4325e+02, 1.9550e+03,
        7.9000e+01, 5.0075e+02, 7.9000e+01, 5.0075e+02, 4.4250e+01,
        2.2450e+02],
       [1.8500e+01, 2.4325e+02, 2.7500e+00, 7.9000e+01, 2.4325e+02,
        1.9550e+03, 7.9000e+01, 5.0075e+02, 2.7500e+00, 7.9000e+01,
        0.0000e+00, 4.4250e+01, 7.9000e+01, 5.0075e+02, 4.4250e+01,
        2.2450e+02],
       [2.2500e+00, 4.8500e+01, 0.0000e+00, 4.4250e+01, 4.8500e+01,
        5.0075e+02, 4.4250e+01, 2.7700e+02, 4.8500e+01, 5.0075e+02,
        4.4250e+01, 2.7700e+02, 5.0075e+02, 9.6050e+02, 2.7700e+02,
        5.2175e+02],
       [1.8500e+01, 2.4325e+02, 2.4325e+02, 1.9550e+03, 2.7500e+00,
        7.9000e+01, 7.9000e+01, 5.0075e+02, 2.7500e+00, 7.9000e+01,
        7.9000e+01, 5.0075e+02, 0.0000e+00, 4.4250e+01, 4.4250e+01,
        2.2450e+02],
       [2.2500e+00, 4.8500e+01, 4.8500e+01, 5.0075e+02, 0.0000e+00,
        4.4250e+01, 4.4250e+01, 2.7700e+02, 4.8500e+01, 5.0075e+02,
        5.0075e+02, 9.6050e+02, 4.4250e+01, 2.7700e+02, 2.7700e+02,
        5.2175e+02],
       [2.2500e+00, 4.8500e+01, 4.8500e+01, 5.0075e+02, 4.8500e+01,
        5.0075e+02, 5.0075e+02, 9.6050e+02, 0.0000e+00, 4.4250e+01,
        4.4250e+01, 2.7700e+02, 4.4250e+01, 2.7700e+02, 2.7700e+02,
        5.2175e+02],
       [0.0000e+00, 1.5750e+01, 1.5750e+01, 2.2450e+02, 1.5750e+01,
        2.2450e+02, 2.2450e+02, 5.2175e+02, 1.5750e+01, 2.2450e+02,
        2.2450e+02, 5.2175e+02, 2.2450e+02, 5.2175e+02, 5.2175e+02,
        5.9900e+02]])

In [73]:
l4 = np.array([[4.18015e+04, 5.12100e+03, 5.12100e+03, 4.70500e+02, 5.12100e+03,
        4.70500e+02, 4.70500e+02, 9.90000e+01, 5.12100e+03, 4.70500e+02,
        4.70500e+02, 9.90000e+01, 4.70500e+02, 9.90000e+01, 9.90000e+01,
        3.15000e+01],
       [5.12100e+03, 4.93500e+02, 4.93500e+02, 1.22000e+02, 4.93500e+02,
        1.22000e+02, 1.22000e+02, 6.75000e+01, 4.15855e+04, 4.31100e+03,
        4.31100e+03, 1.14950e+03, 4.31100e+03, 1.14950e+03, 1.14950e+03,
        5.34000e+02],
       [5.12100e+03, 4.93500e+02, 4.93500e+02, 1.22000e+02, 4.15855e+04,
        4.31100e+03, 4.31100e+03, 1.14950e+03, 4.93500e+02, 1.22000e+02,
        1.22000e+02, 6.75000e+01, 4.31100e+03, 1.14950e+03, 1.14950e+03,
        5.34000e+02],
       [4.70500e+02, 1.22000e+02, 1.22000e+02, 1.35500e+02, 4.31100e+03,
        1.85300e+03, 1.85300e+03, 1.36100e+03, 4.31100e+03, 1.85300e+03,
        1.85300e+03, 1.36100e+03, 2.59215e+04, 7.89400e+03, 7.89400e+03,
        4.22050e+03],
       [5.12100e+03, 4.93500e+02, 4.15855e+04, 4.31100e+03, 4.93500e+02,
        1.22000e+02, 4.31100e+03, 1.14950e+03, 4.93500e+02, 1.22000e+02,
        4.31100e+03, 1.14950e+03, 1.22000e+02, 6.75000e+01, 1.14950e+03,
        5.34000e+02],
       [4.70500e+02, 1.22000e+02, 4.31100e+03, 1.85300e+03, 1.22000e+02,
        1.35500e+02, 1.85300e+03, 1.36100e+03, 4.31100e+03, 1.85300e+03,
        2.59215e+04, 7.89400e+03, 1.85300e+03, 1.36100e+03, 7.89400e+03,
        4.22050e+03],
       [4.70500e+02, 1.22000e+02, 4.31100e+03, 1.85300e+03, 4.31100e+03,
        1.85300e+03, 2.59215e+04, 7.89400e+03, 1.22000e+02, 1.35500e+02,
        1.85300e+03, 1.36100e+03, 1.85300e+03, 1.36100e+03, 7.89400e+03,
        4.22050e+03],
       [9.90000e+01, 6.75000e+01, 1.14950e+03, 1.36100e+03, 1.14950e+03,
        1.36100e+03, 7.89400e+03, 4.89450e+03, 1.14950e+03, 1.36100e+03,
        7.89400e+03, 4.89450e+03, 7.89400e+03, 4.89450e+03, 1.21225e+04,
        7.35000e+03],
       [5.12100e+03, 4.15855e+04, 4.93500e+02, 4.31100e+03, 4.93500e+02,
        4.31100e+03, 1.22000e+02, 1.14950e+03, 4.93500e+02, 4.31100e+03,
        1.22000e+02, 1.14950e+03, 1.22000e+02, 1.14950e+03, 6.75000e+01,
        5.34000e+02],
       [4.70500e+02, 4.31100e+03, 1.22000e+02, 1.85300e+03, 1.22000e+02,
        1.85300e+03, 1.35500e+02, 1.36100e+03, 4.31100e+03, 2.59215e+04,
        1.85300e+03, 7.89400e+03, 1.85300e+03, 7.89400e+03, 1.36100e+03,
        4.22050e+03],
       [4.70500e+02, 4.31100e+03, 1.22000e+02, 1.85300e+03, 4.31100e+03,
        2.59215e+04, 1.85300e+03, 7.89400e+03, 1.22000e+02, 1.85300e+03,
        1.35500e+02, 1.36100e+03, 1.85300e+03, 7.89400e+03, 1.36100e+03,
        4.22050e+03],
       [9.90000e+01, 1.14950e+03, 6.75000e+01, 1.36100e+03, 1.14950e+03,
        7.89400e+03, 1.36100e+03, 4.89450e+03, 1.14950e+03, 7.89400e+03,
        1.36100e+03, 4.89450e+03, 7.89400e+03, 1.21225e+04, 4.89450e+03,
        7.35000e+03],
       [4.70500e+02, 4.31100e+03, 4.31100e+03, 2.59215e+04, 1.22000e+02,
        1.85300e+03, 1.85300e+03, 7.89400e+03, 1.22000e+02, 1.85300e+03,
        1.85300e+03, 7.89400e+03, 1.35500e+02, 1.36100e+03, 1.36100e+03,
        4.22050e+03],
       [9.90000e+01, 1.14950e+03, 1.14950e+03, 7.89400e+03, 6.75000e+01,
        1.36100e+03, 1.36100e+03, 4.89450e+03, 1.14950e+03, 7.89400e+03,
        7.89400e+03, 1.21225e+04, 1.36100e+03, 4.89450e+03, 4.89450e+03,
        7.35000e+03],
       [9.90000e+01, 1.14950e+03, 1.14950e+03, 7.89400e+03, 1.14950e+03,
        7.89400e+03, 7.89400e+03, 1.21225e+04, 6.75000e+01, 1.36100e+03,
        1.36100e+03, 4.89450e+03, 1.36100e+03, 4.89450e+03, 4.89450e+03,
        7.35000e+03],
       [3.15000e+01, 5.34000e+02, 5.34000e+02, 4.22050e+03, 5.34000e+02,
        4.22050e+03, 4.22050e+03, 7.35000e+03, 5.34000e+02, 4.22050e+03,
        4.22050e+03, 7.35000e+03, 4.22050e+03, 7.35000e+03, 7.35000e+03,
        8.64550e+03]])

In [129]:
x

array([[ 1.42931578e+01,  4.20348882e-01,  4.20348882e-01,
         4.46697925e-03,  4.20348882e-01,  4.46697925e-03,
         4.46697925e-03, -6.83534994e-05,  4.20348882e-01,
         4.46697925e-03,  4.46697925e-03, -6.83534994e-05,
         4.46697925e-03, -6.83534994e-05, -6.83534994e-05,
        -1.08174772e-03],
       [ 4.19365258e-01,  1.42794912e+01,  5.50983295e-03,
         4.22893582e-01,  5.50983295e-03,  4.22893582e-01,
         2.35062167e-04,  4.77902703e-03,  5.50983295e-03,
         4.22893582e-01,  2.35062167e-04,  4.77902703e-03,
         2.35062167e-04,  4.77902703e-03, -4.93780013e-03,
         5.82886253e-03],
       [ 4.19365258e-01,  5.50983295e-03,  1.42794912e+01,
         4.22893582e-01,  5.50983295e-03,  2.35062167e-04,
         4.22893582e-01,  4.77902703e-03,  5.50983295e-03,
         2.35062167e-04,  4.22893582e-01,  4.77902703e-03,
         2.35062167e-04, -4.93780013e-03,  4.77902703e-03,
         5.82886253e-03],
       [ 5.29399573e-03,  4.02283370e

In [127]:
x

array([[ 1.41487676e+01,  4.58626761e-01,  4.58626761e-01,
         2.93427230e-03,  4.58626761e-01,  2.93427230e-03,
         2.93427230e-03, -1.89101616e-15,  4.58626761e-01,
         2.93427230e-03,  2.93427230e-03, -3.25627360e-15,
         2.93427230e-03, -1.89101616e-15, -2.38747341e-15,
        -8.80281690e-04],
       [ 4.58626761e-01,  1.41494404e+01,  3.00343565e-03,
         4.58121018e-01,  3.00343565e-03,  4.58121018e-01,
         7.99169856e-04,  1.37039937e-03,  3.00343565e-03,
         4.58121018e-01,  7.99169856e-04,  1.37039937e-03,
         7.99169856e-04,  1.37039937e-03, -4.11119812e-03,
         6.16197183e-03],
       [ 4.58626761e-01,  3.00343565e-03,  1.41494404e+01,
         4.58121018e-01,  3.00343565e-03,  7.99169856e-04,
         4.58121018e-01,  1.37039937e-03,  3.00343565e-03,
         7.99169856e-04,  4.58121018e-01,  1.37039937e-03,
         7.99169856e-04, -4.11119812e-03,  1.37039937e-03,
         6.16197183e-03],
       [ 3.22769953e-03,  4.52347372e

In [117]:
l2

array([[199. ,  13.5,  13.5,   0.5,  13.5,   0.5,   0.5,   0. ,  13.5,
          0.5,   0.5,   0. ,   0.5,   0. ,   0. ,   0. ],
       [ 13.5,   0.5,   0.5,   0. ,   0.5,   0. ,   0. ,   0. , 199. ,
         12.5,  12.5,   1.5,  12.5,   1.5,   1.5,   0. ],
       [ 13.5,   0.5,   0.5,   0. , 199. ,  12.5,  12.5,   1.5,   0.5,
          0. ,   0. ,   0. ,  12.5,   1.5,   1.5,   0. ],
       [  0.5,   0. ,   0. ,   0. ,  12.5,   2.5,   2.5,   0. ,  12.5,
          2.5,   2.5,   0. , 151. ,  29.5,  29.5,  10.5],
       [ 13.5,   0.5, 199. ,  12.5,   0.5,   0. ,  12.5,   1.5,   0.5,
          0. ,  12.5,   1.5,   0. ,   0. ,   1.5,   0. ],
       [  0.5,   0. ,  12.5,   2.5,   0. ,   0. ,   2.5,   0. ,  12.5,
          2.5, 151. ,  29.5,   2.5,   0. ,  29.5,  10.5],
       [  0.5,   0. ,  12.5,   2.5,  12.5,   2.5, 151. ,  29.5,   0. ,
          0. ,   2.5,   0. ,   2.5,   0. ,  29.5,  10.5],
       [  0. ,   0. ,   1.5,   0. ,   1.5,   0. ,  29.5,  14.5,   1.5,
          0. ,  29.5,  14.

In [98]:
l1

array([[14. ,  0.5,  0.5,  0. ,  0.5,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 14. ,  0.5,  0.5,
         0. ,  0.5,  0. ,  0. ,  0. ],
       [ 0.5,  0. ,  0. ,  0. , 14. ,  0.5,  0.5,  0. ,  0. ,  0. ,  0. ,
         0. ,  0.5,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,
         0. , 12. ,  1.5,  1.5,  0. ],
       [ 0.5,  0. , 14. ,  0.5,  0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0.5,
         0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. , 12. ,
         1.5,  0. ,  0. ,  1.5,  0. ],
       [ 0. ,  0. ,  0.5,  0. ,  0.5,  0. , 12. ,  1.5,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  1.5,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1.5,  0. ,  0. ,  0. ,  1.5,
         0. ,  1.5,  0. ,  8. ,  3.5],
       [ 0.5, 14. ,  0. ,  0.5,  0. ,  0.5,  0. ,  0. ,  0. ,  0.5,  0. ,
         0. ,  0. ,  0

### Two gate

In [87]:
l2

array([[199. ,  13.5,  13.5,   0.5,  13.5,   0.5,   0.5,   0. ,  13.5,
          0.5,   0.5,   0. ,   0.5,   0. ,   0. ,   0. ],
       [ 13.5,   0.5,   0.5,   0. ,   0.5,   0. ,   0. ,   0. , 199. ,
         12.5,  12.5,   1.5,  12.5,   1.5,   1.5,   0. ],
       [ 13.5,   0.5,   0.5,   0. , 199. ,  12.5,  12.5,   1.5,   0.5,
          0. ,   0. ,   0. ,  12.5,   1.5,   1.5,   0. ],
       [  0.5,   0. ,   0. ,   0. ,  12.5,   2.5,   2.5,   0. ,  12.5,
          2.5,   2.5,   0. , 151. ,  29.5,  29.5,  10.5],
       [ 13.5,   0.5, 199. ,  12.5,   0.5,   0. ,  12.5,   1.5,   0.5,
          0. ,  12.5,   1.5,   0. ,   0. ,   1.5,   0. ],
       [  0.5,   0. ,  12.5,   2.5,   0. ,   0. ,   2.5,   0. ,  12.5,
          2.5, 151. ,  29.5,   2.5,   0. ,  29.5,  10.5],
       [  0.5,   0. ,  12.5,   2.5,  12.5,   2.5, 151. ,  29.5,   0. ,
          0. ,   2.5,   0. ,   2.5,   0. ,  29.5,  10.5],
       [  0. ,   0. ,   1.5,   0. ,   1.5,   0. ,  29.5,  14.5,   1.5,
          0. ,  29.5,  14.